In [1]:
import os.path as op
import numpy as np
import pandas as pd
import nilearn
import pickle
import os

def load_glm(fpath):
    """Load a GLM object from a file."""
    with open(fpath, 'rb') as f:
       fmri_glm = pickle.load(f)
    return fmri_glm

/home/hyruuk/python_envs/shinobi_env/lib/python3.7/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [2]:
glm_path = '/scratch/hyruuk/neuromod/shinobi2023/processed/glm/ses-level'

glm_filelist = os.listdir(glm_path)

In [3]:
glm_filelist[0]

fmri_glm = load_glm(op.join(glm_path, glm_filelist[0]))

In [4]:
glm_filelist

['sub-04_ses-003_DOWN_simplemodel_fitted_glm.pkl',
 'sub-06_ses-003_UP_simplemodel_fitted_glm.pkl',
 'sub-06_ses-013_LEFT_simplemodel_fitted_glm.pkl',
 'sub-06_ses-013_RIGHT_simplemodel_fitted_glm.pkl',
 'sub-06_ses-004_RIGHT_simplemodel_fitted_glm.pkl',
 'sub-04_ses-010_RIGHT_simplemodel_fitted_glm.pkl',
 'sub-02_ses-002_JUMP_simplemodel_fitted_glm.pkl',
 'sub-01_ses-004_HIT_simplemodel_fitted_glm.pkl',
 'sub-06_ses-003_JUMP_simplemodel_fitted_glm.pkl',
 'sub-02_ses-008_DOWN_simplemodel_fitted_glm.pkl',
 'sub-01_ses-003_Kill_simplemodel_fitted_glm.pkl',
 'sub-02_ses-002_RIGHT_simplemodel_fitted_glm.pkl',
 'sub-01_ses-007_DOWN_simplemodel_fitted_glm.pkl',
 'sub-02_ses-015_RIGHT_simplemodel_fitted_glm.pkl',
 'sub-04_ses-009_JUMP_simplemodel_fitted_glm.pkl',
 'sub-02_ses-012_LEFT_simplemodel_fitted_glm.pkl',
 'sub-04_ses-009_UP_simplemodel_fitted_glm.pkl',
 'sub-01_ses-010_HIT_simplemodel_fitted_glm.pkl',
 'sub-06_ses-006_LEFT_simplemodel_fitted_glm.pkl',
 'sub-06_ses-006_HealthLoss_simp

In [16]:
fmri_glm.

[            DOWN  cosine00  cosine01  cosine02  cosine03  cosine04  cosine05  \
 0.00    0.000000  0.068680  0.068678  0.068676  0.068673  0.068668  0.068663   
 1.49    0.000000  0.068676  0.068663  0.068642  0.068612  0.068574  0.068528   
 2.98    0.000000  0.068668  0.068633  0.068574  0.068492  0.068386  0.068257   
 4.47    0.000000  0.068657  0.068588  0.068473  0.068311  0.068104  0.067851   
 5.96    0.000000  0.068642  0.068528  0.068337  0.068070  0.067728  0.067311   
 ...          ...       ...       ...       ...       ...       ...       ...   
 624.31 -0.002388 -0.068642  0.068528 -0.068337  0.068070 -0.067728  0.067311   
 625.80 -0.001533 -0.068657  0.068588 -0.068473  0.068311 -0.068104  0.067851   
 627.29 -0.000922 -0.068668  0.068633 -0.068574  0.068492 -0.068386  0.068257   
 628.78 -0.000521 -0.068676  0.068663 -0.068642  0.068612 -0.068574  0.068528   
 630.27 -0.000279 -0.068680  0.068678 -0.068676  0.068673 -0.068668  0.068663   
 
         cosine06  cosine0